Basic file reading and data exploration

In [1]:
import h5py
import numpy as np
import math
from scipy.spatial.transform import Rotation as R

# fname = 'd4/20220302_131424.hdf5'
# fname = '20220408_191041.hdf5'
# fname = '20220408_210049.hdf5' # 3 strokes
# fname = '20220409_184944.hdf5' # perpendicular drill angle
fname = '20220409_185037.hdf5' # varying drill angle

f = h5py.File(fname, 'r')
print('File keys', f.keys())
print(type(f))

File keys <KeysViewHDF5 ['burr_change', 'data', 'force', 'metadata', 'voxels_removed']>
<class 'h5py._hl.files.File'>


In [2]:
data = f['data']
print('Data keys', data.keys())
# print((data['depth'][()]).shape)
# print((data['l_img'][()]).shape)
# print((data['pose_main_camera'][()]).shape)
print((data['pose_mastoidectomy_drill'][()]).shape)
# print((data['r_img'][()]).shape)
print((data['segm'][()]).shape)
print((data['time'][()]).shape)

# Not necessary for our purposes

# b_cg = f['burr_change']
# print('Burr Change keys', b_cg.keys())

# meta = f['metadata']
# print('Metadata keys', meta.keys())
# print((meta['README'][()]))

force = f['force']
print('Force keys', force.keys())
print((force['time_stamp'][()]).shape)
print((force['wrench'][()]).shape)

v_rm = f['voxels_removed']
print('Voxels Removed keys', v_rm.keys())
print((v_rm['time_stamp'][()]).shape)
print((v_rm['voxel_color'][()]).shape)
print((v_rm['voxel_removed'][()]).shape)

# # Time differences between either time stamps unequal

# print((data['time'][1]) - (data['time'][0]))
# print((data['time'][435]) - (data['time'][434]))
# print((v_rm['time_stamp'][1]) - (v_rm['time_stamp'][0]))
# print((v_rm['time_stamp'][435]) - (v_rm['time_stamp'][434]))

# print(min(data['time'][()]), max(data['time'][()]))

print(min(data['time'][()]) == min(data['time'][()]))

Data keys <KeysViewHDF5 ['depth', 'l_img', 'pose_main_camera', 'pose_mastoidectomy_drill', 'r_img', 'segm', 'time']>
(100, 7)
(100, 480, 640, 3)
(100,)
Force keys <KeysViewHDF5 ['time_stamp', 'wrench']>
(43882,)
(43882, 3)
Voxels Removed keys <KeysViewHDF5 ['time_stamp', 'voxel_color', 'voxel_removed']>
(20057,)
(20057, 4)
(20057, 3)
True


Function implementation

In [3]:
def get_strokes(stream: np.ndarray, timepts: np.ndarray, k=6):
    stream = stream[:, :3]
    X_P = []

    # Compute k-cosines for each pivot point
    for j,P in enumerate(stream):
        
        # Cannot consider edge points as central k's
        if (j - k < 0) or (j + k >= stream.shape[0]):
            continue

        P_a = stream[j - k]
        P_c = stream[j + k]

        k_cos = np.dot(P_a, P_c) / (np.linalg.norm(P_a) * np.linalg.norm(P_c))
        k_cos = max(min(k_cos, 1), -1)
        X_P.append(180 - (math.acos(k_cos) * (180/np.pi)))

    # Detect pivot points
    mu = np.mean(X_P)
    sig = np.std(X_P)

    for i in range(k):
        
        X_P.insert(0, mu)
        X_P.append(mu)

    X_P = np.array(X_P)

    F_c = [1 if x_P > mu + sig else 0 for x_P in X_P]

    j = 0
    for i in range(1, len(F_c)):

        if F_c[i] == 1 and F_c[i-1] == 0:
            j += 1
        elif sum(F_c[i:i+k]) == 0 and j != 0:
            ind = math.floor(j/2)
            F_c[i-j:i] = [0] * j
            F_c[i-ind] = 1
            j = 0
        elif j != 0:
            j += 1

    st = np.insert(timepts[[s == 1 for s in F_c]], 0, min(timepts))

    return F_c, st

strokes, stroke_times = get_strokes(data['pose_mastoidectomy_drill'][()], data['time'][()])

In [4]:
def stats_per_stroke(stroke_arr: np.ndarray):
    
    mean = np.mean(stroke_arr)
    med_ = np.median(stroke_arr)
    max_ = np.max(stroke_arr)

    return mean, med_, max_

In [5]:
def stroke_force(strokes: np.ndarray, stroke_times: np.ndarray,
                force_stream: np.ndarray, force_times: np.ndarray):

    avg_stroke_force = []
    for i in range(sum(strokes)):

        stroke_mask = [ft >= stroke_times[i] and ft < stroke_times[i+1] for ft in force_times]
        stroke_forces = np.linalg.norm(force_stream[stroke_mask], axis=1)
        avg_stroke_force.append(np.mean(stroke_forces))

    return np.array(avg_stroke_force)
    

stroke_force(strokes, stroke_times, force['wrench'][()], force['time_stamp'][()])

array([0.23464485, 0.61514681, 0.47998663])

In [6]:
def stroke_length(strokes: np.ndarray, stream: np.ndarray):

    stream = stream[:, :3]
    
    lens = []
    inds = np.insert(np.where(strokes == 1), 0, 0)
    for i in range(sum(strokes)):
        stroke_len = 0
        curr_stroke = stream[inds[i]:inds[i+1]]
        for j in range(1, len(curr_stroke)):

            stroke_len += np.linalg.norm(curr_stroke[j-1] - curr_stroke[j])

        lens.append(stroke_len)

    return np.array(lens)

stroke_length(np.array(strokes), data['pose_mastoidectomy_drill'][()])

array([0.12272328, 0.11159843, 0.0228806 ])

In [7]:
def bone_removal_rate(strokes: np.ndarray, stroke_times: np.ndarray,
                        stream: np.ndarray, voxel_times: np.ndarray):
    
    vox_rm = []
    for i in range(sum(strokes)):

        stroke_voxels = [vt >= stroke_times[i] and vt < stroke_times[i+1] for vt in voxel_times]
        vox_rm.append(sum(stroke_voxels))

    rate = np.divide(np.array(vox_rm), stroke_length(np.array(strokes), stream))

    return rate

bone_removal_rate(strokes, stroke_times, data['pose_mastoidectomy_drill'][()], v_rm['time_stamp'][()])

array([ 18855.4288647 ,  96461.92971425, 250212.00467123])

In [8]:
def procedure_duration(timepts: np.ndarray):
    return (max(timepts) - min(timepts))

procedure_duration(data['time'][()])

43.95865249633789

In [9]:
def drill_orientation(stream: np.ndarray, timepts: np.ndarray,
                      voxels: np.ndarray, voxel_times: np.ndarray):
    
    angles = []
    for i,t in enumerate(timepts):
        ind = np.argmin(np.abs(voxel_times - t))
        if np.isclose(np.abs(voxel_times[ind] - t), 0):
            # print(sum([np.isclose(np.abs(v - t), 0) for v in voxel_times]))
            # if sum([np.isclose(v - t, 0) for v in voxel_times]) >= 3:
            #     print(ind)
            #     print('howdy')
            #     return

            # Just use three closest
            ind2 = 0
            ind3 = 0
            if ind < len(voxel_times) - 2:
                ind2 = ind+1
                ind3 = ind+2
            elif ind > 1:
                ind2 = ind-1
                ind3 = ind-2

            x0, y0, z0 = voxels[ind]
            x1, y1, z1 = voxels[ind2]
            x2, y2, z2 = voxels[ind3]

            ux, uy, uz = [x1-x0, y1-y0, z1-z0]
            vx, vy, vz = [x2-x0, y2-y0, z2-z0]

            u_cross_v = [uy*vz-uz*vy, uz*vx-ux*vz, ux*vy-uy*vx]
            normal = np.array(u_cross_v)
            # print(normal)
            normal = np.divide(normal, np.linalg.norm(normal))
            # print(R.from_quat(stream[i, 3:]).inv().apply(normal))

            v = R.from_quat(stream[i, 3:]).apply([0, 1, 0.5])
            v = np.divide(v, np.linalg.norm(v))
            # print(v)
            # print(voxels[ind])

            angle = np.arccos(np.clip(np.dot(v, normal), -1.0, 1.0)) * (180/np.pi)
            if angle > 90:
                angle = 180 - angle
            angles.append(90 - angle)

    print(np.mean(angles))
    return angles

drill_orientation(data['pose_mastoidectomy_drill'][()], data['time'][()], v_rm['voxel_removed'][()], v_rm['time_stamp'][()])

28.689006273632835


[25.54635818587745,
 20.14645994662682,
 9.335613144616659,
 44.053876184728395,
 6.176036847103546,
 12.956087376926419,
 54.384082086796354,
 7.5409324423904,
 39.73540785162993,
 29.939695216746742,
 68.63760440357333,
 65.76027871718426,
 12.905140303660787,
 33.83526152658561,
 28.92871288732396,
 61.44644971508029,
 54.175627593654525,
 32.599772537673985,
 10.17910461146441,
 49.87239389955912,
 9.425798421241552,
 43.351086290617715,
 45.12050417293061,
 22.966739818060063,
 11.612575509173155,
 1.21415240874569,
 17.0208234730936,
 3.2655528277877863,
 34.5236899790975,
 29.50537673042242,
 25.837990924844703,
 14.66226286855236,
 18.797083608491576,
 8.325542759840332,
 8.066937632860515,
 45.4472570114344,
 7.622922806829919,
 14.712989908010329,
 35.444604886674924,
 12.101214004607755,
 74.98183513290186,
 52.240025922532396,
 14.222934632485135,
 17.103230922914335,
 23.671689545997154,
 60.29457090775958]